# First/Every-visit MC prediction 


 
(c) Fabrice Mulotti

In [4]:
import gymnasium as gym
import pandas as pd
from collections import defaultdict

Create a blackjack environment:

In [5]:
env = gym.make('Blackjack-v1')

## Définissons une politque

state[0] renvoie le nombre de point du joueur.   

L'action :      
0 est standby   
1 est hit (nouvelle carte)  

In [9]:
def policy(pstate):
    # une ligne : définir une politique qui choisie l'action stand (0) si le joueur à 19 points ou moins, sinon hit (1)
    # le nombre de points est à l'indice
    return 0 if pstate[0] >19 else 1

In [10]:
# test !
assert policy([19,0,0])==1

In [11]:
# test !
assert policy([20,0,0])==0

In [12]:
## créons notre environnement
state = env.reset()
print(state)

((18, 4, 0), {})


In [13]:
print(policy(state[0]))

1


In [14]:
# Enregistrement des épisodes
episode = []
    
# Remise à zéro env
ge_state = env.reset()[0]

In [15]:
ge_state


(13, 10, 0)

In [16]:
action = policy(ge_state)
action

1

***
## Generatrice d'épisode
Notre prochain fonction va nous permettre de générer des épisodes


In [43]:
def generate_episode(policy,max_turn):
    
    # Enregistrement des épisodes
    episode = []
    
    # Remise à zéro env
    ge_state = env.reset()[0]
  
    # déroulement de la partie
    for i in range(max_turn):
        # choix d'une action en fonction de notre police et l'étar
        # 1 ligne
        action = policy(ge_state)
        # on joue notre action en on récupère les résultats
        ge_next_state, reward, done, truncated, info = env.step(action)
        
        # enregistrement
        # print(f'Next state {ge_next_state}')
        episode.append((ge_state, action, reward))
        
        # Condition de fin ?
        if done or truncated:
            break

        # sinon la partie continue
        ge_state = ge_next_state

    return episode

In [36]:
# Devrait produit une partie
episode = generate_episode(policy,100)
episode

Next state (21, 3, 1)


[((21, 3, 1), 0, 1.0)]

***
## Calcul de la fonction de valeur

Nous allons devoir générer un grand nombre d'épisode et ajuster la `valeur moyenne de v` à chaque passage sur un état s

 

In [32]:
# total_return = fonction de valeur pour chaque état
total_return = defaultdict(float)

# N nombre de passage sur chaque état
N = defaultdict(int)

In [33]:
# Nombre d'itération
#num_iterations = 50000
num_iterations =2
gamma=0.90

In [ ]:
print_debug = True

for i in range(num_iterations):
    
    # générons un épisode    
    # générer un episode et sauvegarder dans la variable 'episode'
    episode = generate_episode(policy,num_iterations)
    
    # stockons l'état, action et récompense obtenue / zip permet de prendre chaque 1, 2eme, 3eme.. terme
    states, actions, rewards = zip(*episode)

    if(print_debug):
        print(f'states : {states}')
        print(f'actions : {actions}')
        print(f'rewards : {rewards}')
    
    # pour chaque état rencontré dans l'épisode
    for state in episode:
        if(print_debug):
            print(f'state : {state}')

    # calculer pour chaque état traversé, le gain G de l'état et mettre à jour les variables total_return[s] et N[s]
    G=0 
    for t in range(len(states)-1,-1,-1):
        print(f't: {t}')
        # 4-6 lignes
        state = states[t]
        G = rewards[t] + gamma*G
        total_return[state] +=G
        N[state] +=1
        
            

states : ((19, 6, 0),)
actions : (1,)
rewards : (-1.0,)
state : ((19, 6, 0), 1, -1.0)
t: 0
states : ((9, 5, 0), (19, 5, 0))
actions : (1, 1)
rewards : (0.0, -1.0)
state : ((9, 5, 0), 1, 0.0)
state : ((19, 5, 0), 1, -1.0)
t: 1
t: 0
states : ((14, 8, 0), (21, 8, 0))
actions : (1, 0)
rewards : (0.0, 1.0)
state : ((14, 8, 0), 1, 0.0)
state : ((21, 8, 0), 0, 1.0)
t: 1
t: 0
states : ((20, 6, 0),)
actions : (0,)
rewards : (1.0,)
state : ((20, 6, 0), 0, 1.0)
t: 0
states : ((20, 1, 0),)
actions : (0,)
rewards : (0.0,)
state : ((20, 1, 0), 0, 0.0)
t: 0
states : ((14, 10, 0), (21, 10, 0))
actions : (1, 0)
rewards : (0.0, 1.0)
state : ((14, 10, 0), 1, 0.0)
state : ((21, 10, 0), 0, 1.0)
t: 1
t: 0
states : ((21, 7, 1),)
actions : (0,)
rewards : (1.0,)
state : ((21, 7, 1), 0, 1.0)
t: 0
states : ((10, 2, 0), (20, 2, 0))
actions : (1, 0)
rewards : (0.0, -1.0)
state : ((10, 2, 0), 1, 0.0)
state : ((20, 2, 0), 0, -1.0)
t: 1
t: 0
states : ((12, 1, 0), (17, 1, 0))
actions : (1, 1)
rewards : (0.0, -1.0)
sta

***
## Analysons nos résultats
Convertissons les données en dataframe pour une lecture plus facile

In [ ]:
total_return = pd.DataFrame(total_return.items(),columns=['state', 'total_return'])

In [ ]:
N = pd.DataFrame(N.items(),columns=['state', 'N'])

Merge des deux dataframe

In [ ]:
df = pd.merge(total_return, N, on="state")

Résultat : 

In [ ]:
df.head(10)

Calculons la fonction de valeur.
A ce stade nous avons tous les éléments (cumul des G et nombre de passage N)

In [ ]:
df['value'] = df['total_return']/df['N']

In [ ]:
df.head(20)

***
## Analyse

A ce stade nous pouvons apprécier la valeur de notre politique (seuil à 19).<br>
Exemple dans une situation ou nous avons 21 points.



In [ ]:
# sans AS
df[df['state']==(21,10,False)]['value'].values

In [ ]:
# avec AS
df[df['state']==(21,10,True)]['value'].values

La fonction de valeur est proche de 1. Donc efficiente.



---
Regardons maintenant dans un contexte plus difficile avec une main prochain de 19 (voir politique)

In [ ]:
df[df['state']==(19,8,False)]['value'].values

In [ ]:
df[df['state']==(19,8,True)]['value'].values

La valeur est moindre car l'incertitude de la suite de la partie plus grande